In [14]:
import json
import pandas as pd

file_path = "historicalData/historical_data.json"
data = []
with open(file_path, 'r') as file:
    data = json.load(file)
    # Extract last nodes with datetime as index
    

In [15]:
last_nodes = []
for item in data[:6]:
    timestamp, values = list(item.items())[0]
    #print(timestamp, values)
    if values:
        for value in values:
            value["timestamp"] = pd.to_datetime(value["datetime"])
            last_nodes.append(value)

df = pd.DataFrame(last_nodes).set_index('timestamp')

In [17]:
df

,close,datetime,exchange_code,expiry_date,high,low,open,open_interest,product_type,right,stock_code,strike_price,volume
timestamp,,,,,,,,,,,,,
2019-01-25 09:15:00,124.75,2019-01-25 09:15:00,NFO,31-JAN-2019,127.35,107.50,111.85,424950,Options,Call,NIFTY,10850.0,48452
2019-01-25 09:16:00,124.30,2019-01-25 09:16:00,NFO,31-JAN-2019,126.00,123.15,124.65,424950,Options,Call,NIFTY,10850.0,29700
2019-01-25 09:17:00,124.90,2019-01-25 09:17:00,NFO,31-JAN-2019,126.00,122.85,124.50,421425,Options,Call,NIFTY,10850.0,39450
2019-01-25 09:18:00,128.15,2019-01-25 09:18:00,NFO,31-JAN-2019,129.25,124.30,125.00,421425,Options,Call,NIFTY,10850.0,21975
2019-01-25 09:19:00,129.30,2019-01-25 09:19:00,NFO,31-JAN-2019,130.00,127.40,127.75,421425,Options,Call,NIFTY,10850.0,22800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-28 15:25:00,31.05,2019-01-28 15:25:00,NFO,31-JAN-2019,31.45,30.90,31.00,3923100,Options,Call,NIFTY,10800.0,93975
2019-01-28 15:26:00,31.60,2019-01-28 15:26:00,NFO,31-JAN-2019,31.60,30.80,30.80,3842775,Options,Call,NIFTY,10800.0,149175
2019-01-28 15:27:00,33.45,2019-01-28 15:27:00,NFO,31-JAN-2019,33.55,31.40,31.40,3842775,Options,Call,NIFTY,10800.0,154050


In [18]:
# Group data by day
daily_data = df.groupby(pd.to_datetime(df['datetime']).dt.date)

# Shift the 'close' column within each daily group
shifted_y = daily_data['close'].shift(-future_steps)

# Gather the last t steps of feature data into one row for each day
t_steps_data = []
for _, group in daily_data:
    rolled_features = group[['open', 'open_interest', 'volume']].rolling(window=t, min_periods=1)
    flattened_features = rolled_features.apply(lambda x: x.values.flatten() if isinstance(x, pd.DataFrame) and not np.isnan(x.values).any() else np.nan, raw=True)
    if not flattened_features.empty:
        t_steps_data.append(flattened_features.iloc[-1])
    else:
        t_steps_data.append([np.nan] * (3 * t))  # Fill with NaNs if no valid data

# Concatenate the gathered feature data back to the original dataframe
t_steps_df = pd.DataFrame(t_steps_data, columns=[f'prev_{i}_{col}' for i in range(1, t+1) for col in ['open', 'open_interest', 'volume']])
df = pd.concat([df.reset_index(drop=True), t_steps_df.reset_index(drop=True), shifted_y.reset_index(drop=True)], axis=1)

# Drop rows with NaN values (introduced by shifting or rolling window)
df.dropna(inplace=True)

# Features and target variables
X_cols = [f'prev_{i}_{col}' for i in range(1, t+1) for col in ['open', 'open_interest', 'volume']]
X = df[X_cols]
y = df['close'].shift(-future_steps)  # Future close value shifted by future_steps


In [19]:
shifted_y

timestamp
2019-01-25 09:15:00    128.15
2019-01-25 09:16:00    129.30
2019-01-25 09:17:00    137.30
2019-01-25 09:18:00    136.00
2019-01-25 09:19:00    132.90
                        ...  
2019-01-28 15:25:00     33.70
2019-01-28 15:26:00     32.05
2019-01-28 15:27:00       NaN
2019-01-28 15:28:00       NaN
2019-01-28 15:29:00       NaN
Name: close, Length: 746, dtype: float64